# Neural Network from First Principles

This is an example of a simple 2-layer neural network constructed from first principles using Numpy, which will perform a binary classification task.

The input size is the number of pixels in a $64 x 64 = 4096$ image.  The size of the hidden layer is determined by the hyperparameter `n_h`, and the size of the output layer is $1$ to facilitate the binary classification capability of the model.

Import utils package and load data

In [13]:
from Neural_Networks.utils import *

%matplotlib inline
np.random.seed(1)

# Load data
#Since data is in n x m format, convert into m x n format, m: sample size, n: number of features
X_train_orig, y_train_orig, X_test_orig, y_test_orig = load_data()
X_train = X_train_orig.T
y_train = y_train_orig.T
X_test = X_test_orig.T
y_test = y_test_orig.T

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(286, 4096)
(286, 1)
(125, 4096)
(125, 1)


# Initialize Parameters

The parameters associated with the hidden layer are $W^{[1]}$ and $b^{[1]}$, and the parameters associated with the output layer are $W^{[2]}$ and $b^{[2]}$.

We use **tanh** as the activation function for hidden layer, and **sigmoid** for the output layer.

In [14]:
def init_params(n_i, n_h, n_o):
    """
    Args:
    n_i -- size of input layer
    n_h -- size of hidden layer
    n_o -- size of output layer

    Return:
    params -- a dict object containing all parameters:
        W1 -- weight matrix of layer 1
        b1 -- bias vector of layer 1
        W2 -- weight matrix of layer 2
        b2 -- bias vector of layer 2
    """
    np.random.seed(2) # For deterministic repeatability

    # hidden layer parameters
    W1 = np.random.randn(n_h, n_i) * (0.01)
    b1 = np.zeros((1, n_h))

    # output layer parameters
    W2 = np.random.randn(n_o, n_h) * (0.01)
    b2 = np.zeros((1, n_o))

    params = {'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}


    return params

ps = init_params(3, 4, 1)
print('W1 =', ps['W1'])
print('b1 =' ,ps['b1'])
print('W2 =', ps['W2'])
print('b2 =', ps['b2'])

W1 = [[-0.00416758 -0.00056267 -0.02136196]
 [ 0.01640271 -0.01793436 -0.00841747]
 [ 0.00502881 -0.01245288 -0.01057952]
 [-0.00909008  0.00551454  0.02292208]]
b1 = [[0. 0. 0. 0.]]
W2 = [[ 0.00041539 -0.01117925  0.00539058 -0.0059616 ]]
b2 = [[0.]]


# Forward Propagation

The following formulas are used to implement forward propagation:
- $z^{[1]} = XW^{[1]T} + b^{[1]}$
- $a^{[1]} = tanh(z^{[1]})$
- $z^{[2]} = a^{[1]}W^{[2]T} + b^{[2]}$
- $z^{[2]} = \sigma(z^{[2]})$

In [15]:
def forward_prop(X, params):
    """
    Args:
    X -- input data of shape (m,n_in)
    params -- a python dict object containing all parameters (output of init_params)

    Return:
    a2 -- the activation of the output layer
    cache -- a python dict containing all intermediate values for later use in backprop
             i.e., 'z1', 'a1', 'z2', 'a2'
    """
    m = X.shape[0]

    # Retrieve parameters from params
    W1 = np.array(params['W1'])
    b1 = np.array(params['b1'])

    W2 = np.array(params['W2'])
    b2 = np.array(params['b2'])

    # Implement forward propagation
    z1 = np.dot(X, W1.T) + b1
    a1 = np.tanh(z1)

    z2 = np.dot(a1, W2.T) + b2
    a2 = sigmoid(z2)

    assert a1.shape[0] == m
    assert a2.shape[0] == m

    cache = {'z1': z1, 'a1': a1, 'z2': z2, 'a2': a2}

    return a2, cache

X_tmp, params_tmp = forwardprop_testcase()

a2, cache = forward_prop(X_tmp, params_tmp)

print('mean(z1) =', np.mean(cache['z1']))
print('mean(a1) =', np.mean(cache['a1']))
print('mean(z2) =', np.mean(cache['z2']))
print('mean(a2) =', np.mean(cache['a2']))

mean(z1) = 0.006415781628350418
mean(a1) = 0.006410368144939439
mean(z2) = -6.432516196270971e-05
mean(a2) = 0.49998391870952374


# Backward Propagation

The following formulas are used to implement backward propagation:
- $dz^{[2]} = a^{[2]} - y$
- $dW^{[2]} = \frac{1}{m}dz^{[2]T}a^{[1]}$ --> where $m$ is the number of examples
- $db^{[2]} = \frac{1}{m}$ np.sum( $dz^{[2]}$, axis=0, keepdims=True)
- $da^{[1]} = dz^{[2]}W^{[2]}$
- $dz^{[1]} = da^{[1]}*g'(z^{[1]})$
    - Here $*$ denotes element-wise multiplication
    - $g(z)$ is the tanh function, therefore its derivative is $g'(z^{[1]}) = 1 - (g(z^{[1]}))^2 = 1 - (a^{[1]})^2$
- $d{W}^{[1]} = \frac{1}{m} d{z}^{[1]T}X$
- $d{b}^{[1]} = \frac{1}{m}$ np.sum( $d{z}^{[1]}$, axis=0, keepdims=True)

In [16]:
def backward_prop(X, y, params, cache):
    """
    Args:
    X -- input data of shape (m,n_in)
    y -- input label of shape (m,1)
    params -- a python dict containing all parameters
    cache -- a python dict containing 'Z1', 'A1', 'Z2' and 'A2' (output of forward_prop)

    Return:
    grads -- a python dict containing the gradients w.r.t. all parameters,
             i.e., dW1, db1, dW2, db2
    """
    m = X.shape[0]

    # Retrieve parameters
    W1 = np.array(params['W1'])
    b1 = np.array(params['b1'])
    W2 = np.array(params['W2'])
    b2 = np.array(params['b2'])

    # Retrieve intermediate values stored in cache
    z1 = np.array(cache['z1'])
    a1 = np.array(cache['a1'])
    z2 = np.array(cache['z2'])
    a2 = np.array(cache['a2'])

    # Implement backprop
    dz2 = a2 - y
    dW2 = (1 / m) * np.dot(dz2.T, a1)
    db2 = (1 / m) * np.sum(dz2, axis=0, keepdims=True)
    da1 = np.dot(dz2, W2)
    dz1 = np.multiply(da1, (1 - a1**2))
    dW1 = (1 / m) * np.dot(dz1.T, X)
    db1 = (1 / m) * np.sum(dz1, axis=0, keepdims=True)

    grads = {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}

    return grads

X_tmp, y_tmp, params_tmp, cache_tmp = backprop_testcase()

grads = backward_prop(X_tmp, y_tmp, params_tmp, cache_tmp)
print('mean(dW1)', np.mean(grads['dW1']))
print('mean(db1)', np.mean(grads['db1']))
print('mean(dW2)', np.mean(grads['dW2']))
print('mean(db2)', np.mean(grads['db2']))

mean(dW1) -0.00014844465852477853
mean(db1) -0.0002838378969105248
mean(dW2) -0.004079186018202939
mean(db2) 0.09998392000000002


# Update Parameters

Update $W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}$ as follows:
- $W^{[1]} = W^{[1]} - \alpha\ dW^{[1]}$
- $b^{[1]} = b^{[1]} - \alpha\ db^{[1]}$
- $W^{[2]} = W^{[2]} - \alpha\ dW^{[2]}$
- $b^{[2]} = b^{[2]} - \alpha\ db^{[2]}$

In [17]:
def update_params(params, grads, alpha):
    """
    Args:
    params -- a python dict containing all parameters
    grads -- a python dict containing the gradients w.r.t. all parameters (output of backward_prop)
    alpha -- learning rate

    Return:
    params -- a python dict containing all updated parameters
    """

    # Retrieve parameters
    W1 = np.array(params['W1'])
    b1 = np.array(params['b1'])
    W2 = np.array(params['W2'])
    b2 = np.array(params['b2'])

    # Retrieve gradients
    dW1 = np.array(grads['dW1'])
    db1 = np.array(grads['db1'])
    dW2 = np.array(grads['dW2'])
    db2 = np.array(grads['db2'])

    # Update each parameter
    params['W1'] = W1 - (alpha * dW1)
    params['b1'] = b1 - (alpha * db1)
    params['W2'] = W2 - (alpha * dW2)
    params['b2'] = b2 - (alpha * db2)

    return params

params_tmp, grads_tmp = update_params_testcase()

params = update_params(params_tmp, grads_tmp, 0.01)
print('W1 =', params['W1'])
print('b1 =', params['b1'])
print('W2 =', params['W2'])
print('b2 =', params['b2'])

W1 = [[ 0.004169   -0.00056367 -0.02136304]
 [ 0.0163645  -0.01790747 -0.00838857]
 [ 0.00504726 -0.01246588 -0.01059348]
 [-0.00911046  0.0055289   0.0229375 ]]
b1 = [[-4.13852251e-07  1.12173654e-05 -5.39304763e-06  5.94305036e-06]]
W2 = [[ 0.00048642 -0.011058    0.00546531 -0.00606545]]
b2 = [[-0.00099984]]


# Integrated Model

In [18]:
def nn_model(X, y, n_h, num_iters=10000, alpha=0.01, verbose=False):
    """
    Args:
    X -- training data of shape (m,n_in)
    y -- training label of shape (m,1)
    n_h -- size of hidden layer
    num_iters -- number of iterations for gradient descent
    verbose -- print cost every 1000 steps

    Return:
    params -- parameters learned by the model. Use these to make predictions on new data
    """
    np.random.seed(3)
    m = X.shape[0]
    n_in = X.shape[1]
    n_out = 1

    # Initialize parameters and retrieve them
    params = init_params(n_in, n_h, n_out)

    cost = 0 # added to allow usage of cost outside for loop
    # Gradient descent loop
    for i in range(num_iters):
        # Forward propagation
        y_hat, model_cache = forward_prop(X, params)

        # Backward propagation
        gradients = backward_prop(X, y, params, model_cache)

        # Update parameters
        params = update_params(params, gradients, alpha)

        # Compute cost
        cost = -(1 / m) * np.sum(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))

        # Print cost
        if i % 1000 == 0 and verbose:
            print('Cost after iter {}: {}'.format(i, cost))

    # print cost of final iteration
    print('Cost after iter {}: {}'.format(num_iters, cost))

    return params

X_tmp, y_tmp = nn_model_testcase()

params_tmp = nn_model(X_tmp, y_tmp, n_h=5, num_iters=5000, alpha=0.01)
# params_tmp = nn_model(X_tmp, y_tmp, n_h=5, num_iters=5000, alpha=0.01, verbose=True)
print('W1 =', params_tmp['W1'])
print('b1 =', params_tmp['b1'])
print('W2 =', params_tmp['W2'])

Cost after iter 5000: 0.018944299307132376
W1 = [[ 0.30237531 -0.17417915 -0.15306611]
 [ 1.25575279 -0.42239646 -0.35147978]
 [ 1.29886467 -0.43536728 -0.36668058]
 [-1.32065465  0.43563934  0.37269501]
 [ 0.41146082 -0.22524765 -0.15315463]]
b1 = [[-0.10251157 -0.82319548 -0.85962928  0.87045666 -0.16520153]]
W2 = [[ 0.42009393  1.87265216  1.95145175 -1.98319859  0.56655482]]


# Use Model to Make Prediction

In [19]:
def predict(X, params):
    """
    Args:
    X -- input data of shape (m,n_in)
    params -- a python dict containing the learned parameters

    Return:
    pred -- predictions of model on X, a vector of 0s and 1s
    """

    a2, _ = forward_prop(X, params)
    pred = np.where(a2 > 0.5, 1.0, 0.0)

    return pred

pred = predict(X_tmp, params_tmp)
print('predictions = ', pred)

predictions =  [[0.]
 [1.]
 [1.]
 [0.]
 [0.]]


# Train and Evaluate Model

In [20]:
# Train the model on X_train and y_train, and print cost

params = nn_model(X_train, y_train, n_h = 10, num_iters=10000, verbose=True)
# params = nn_model(X_train, y_train, n_h = 10, num_iters=2000, verbose=True)

# Make predictions on X_test
pred = predict(X_test, params)


# Compute accuracy, acc, by comparing predictions and y_test

# true positive: predicted positive when actually positive
TP = np.sum(np.where((pred == 1) & (y_test == 1), 1, 0))
# false positive: predicted positive when actually negative
FP = np.sum(np.where((pred == 1) & (y_test == 0), 1, 0))
# true negative: predicted negative when actually negative
TN = np.sum(np.where((pred == 0) & (y_test == 0), 1, 0))
# false negative: predicted negative when actually positive
FN = np.sum(np.where((pred == 0) & (y_test == 1), 1, 0))

acc = (TP + TN) / (TP + FP + TN + FN)

print('Accuracy = {0:.2f}%'.format(acc * 100))

Cost after iter 0: 0.6931077265775999
Cost after iter 1000: 0.2720475478319349
Cost after iter 2000: 0.05469637887688324
Cost after iter 3000: 0.02430101784614092
Cost after iter 4000: 0.014579972250089317
Cost after iter 5000: 0.010120587793618466
Cost after iter 6000: 0.007636411687846548
Cost after iter 7000: 0.006077673661571548
Cost after iter 8000: 0.005018451353041394
Cost after iter 9000: 0.004256522258860162
Cost after iter 10000: 0.0036851156885626913
Accuracy = 95.20%
